## SIMESH DEMO: INTRODUCTION AND MAIN FUNCS DISPLAY

### Data Preparations 

To reduce the size of the module, we do not include the datfile for display, but make a new AMRDataSet directly here with the TDm model constructed by the RBSL model (Titov et al, 2014; Titov et al. 2020).

In [ ]:
# !!! It may take a while to initiate the TDm model for a large domain !!!

import numpy as np
import matplotlib.pyplot as plt
from simesh import configurations, load_from_uarrays, amr_loader

# prepare necessary parameters for initialization of a new AMRDataSet
xmin = np.array([-4,-4,0])
xmax = np.array([4,4,8])
domain_nx = np.array([30,30,30])
block_nx = np.array([10,10,10])

# parameters for the TDm model
r0 = 1.5
a0 = 0.3
q0 = 1.0
L0 = 1.0
d0 = 0.5
naxis = 500
ispositive = True

# prepare the mesh data of domain size
rho = np.ones(domain_nx) # uniform density just for display, no physical meaning
v1 = np.zeros(domain_nx)
v2 = np.zeros(domain_nx)
v3 = np.zeros(domain_nx)
TDm_slab = configurations.TDm_slab(xmin, xmax, domain_nx, r0, a0, ispositive, naxis, q0, L0, d0) # magnetic field b1,b2,b3 from TDm model

w_arrays = np.stack([rho, v1, v2, v3, TDm_slab[2], TDm_slab[1], TDm_slab[0]], axis=-1)

# display the b3 at z=0
im1 = plt.imshow(np.rot90(TDm_slab[2,:,:,0]))
cb1 = plt.colorbar(label='Bz at z=0')

w_names = ['rho', 'v1', 'v2', 'v3', 'b1', 'b2', 'b3']

# here you can specify a datfile name for further output like: file_path='example.dat'; and any other parameters by **kwargs to update the default header
# you can view the default header by: simesh.header_template
ds = load_from_uarrays(w_arrays, w_names, xmin, xmax, block_nx, file_path='data/tdm.dat') 

### Interfaces of the AMRDataSet

If you here prefer to use the datfile from AMRVAC, you can use the following interfaces:

In [65]:
# first output the datfile to prepare the datfile (same as the datfile from AMRVAC)
ds.write_datfile()

# then read from the datfile just for display, same for those from AMRVAC
ds = amr_loader('data/tdm.dat')

assert np.all(ds.mesh.udata == w_arrays), "The data read from the datfile is not consistent with the data used to initialize the AMRDataSet"

the header is a dictionary, it is same as the header in the datfile containing the crucial information of the mesh and simulation parameters:

In [ ]:
ds.header

since it is a slab domain, we preserve the slab version of the mesh data with the shape same as the input data:

In [ ]:
ds.mesh.udata.shape

for the AMR structured mesh data, it is stored ordered by the Morton order with shape:

In [ ]:
ds.mesh.data.shape

in which 27 is the number of blocks (3x3x3 here), (14,14,14) is (10,10,10) (block_nx) + 2 (nghostcells) * 2 (both sides), 7 is the number of physical variables

we can get the information (indices; level; morton_number[=sfc_index+1]; is_leaf) of each block in the 27 blocks by the sfc index [0-26]:

Here the indices [ig] is the index of the block in 3x3x3 level1 blocks array

In [ ]:
ds.mesh.forest.sfc_to_node[2].node # note that ig is derived from node.ig1, node.ig2, node.ig3 

In [ ]:
# rnode is the node attribute of the leaf node
# xmin, ymin, zmin, xmax, ymax, zmax, dx, dy, dz for each element
ds.mesh.rnode[:,0]